In [1]:
'065676'

'065676'

# Imports

In [1]:
import backtrader
from helpers.datahandler import *
from helpers.investing import *
from helpers.journal_handler import *
present = date.today()

# Defaults

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

In = Investing()

private_key = join(expanduser('~'),'Documents','client_secret.json')
JH = JournalHandler(private_key)

journal = JH.get_journal('Finance Journal')

--------------------------------------------------------------------------- 

Trailing Stop Loss: 

HDFC bought on: 2021-08-11 00:00:00,

--------------------------------------------------------------------------- 



In [4]:
JH.total_pl(journal)

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


-268.76

# Swing
## All Details

In [75]:
budget = 15000

df = In.get_recent_info(nifty = 100, **{'mvs':[20,50,100,200]})
df = df[(df['Overbought'] == False) & (df['Ichi Count'] > 0) & 
        (df['Over 50-SMA'] == True) & (df['Over 100-SMA'] == True) & (df['Over 200-SMA'] == True) &
       (df['Momentum ADX'] > 25) & 
        (df['LTP'] <= budget)]

df.head(2)

,Date,Name,LTP,Index,Over 20-SMA,Over 50-SMA,Over 100-SMA,Over 200-SMA,Overbought,Oversold,Momentum ADX,Ichi Count,1 Candle,2 Candles,3 Candles
0,2021-09-09,ACC,2458.75,Nifty 100,True,True,True,True,False,False,30.94696,2,Unknown Green,Unknown,Unknown
3,2021-06-08,ADANIGREEN,1267.55,Nifty 100,True,True,True,True,False,False,26.17697,2,Unknown Red,Unknown,Unknown


## 44-MA
`1:2` RRR

In [72]:
budget = 15000
risk = 150
df = In.show_full_stats(budget=budget,risk=risk, diff = risk, nifty='nifty_50')
df = df[(df['Expected Change %'] <= 15) & (df['RSI']<70) & (df['Direction'] != 'Falling Low')]
df

,DATE,OPEN,HIGH,LOW,CLOSE,52W H,52W L,SYMBOL,Diff,Direction,Ichi,RSI,ATR,Triple Candle,Double Candle,Recent Candle,Index,Expected Change %,Max Config Risk
7,2021-09-09,294.00,303.50,293.55,298.95,360.75,122.15,TATAMOTORS,1.34,Undeterministic,1,53.534040,8.089286,Unknown,Unknown,Unknown Green,Nifty 50,8.22,149.94
9,2021-09-09,119.10,123.80,118.20,122.15,128.50,64.10,ONGC,1.49,Rising High,1,60.225213,3.203571,V Pattern,Unknown,Unknown Green,Nifty 50,10.55,143.91
4,2021-09-09,211.05,213.90,210.55,212.65,239.20,163.35,ITC,2.20,Undeterministic,0,58.859878,2.921429,V Pattern,Bullish Engulfing,Unknown Green,Nifty 50,4.32,147.96
15,2021-09-09,430.25,433.80,430.00,432.30,467.45,175.50,SBIN,2.40,Undeterministic,0,56.103047,7.360714,Unknown,Unknown,Unknown Green,Nifty 50,3.37,146.22
11,2021-09-09,112.65,114.65,112.40,112.80,117.85,71.65,IOC,5.80,Rising High,1,64.848409,2.271429,Unknown,Unknown,Inverted Hammer,Nifty 50,6.58,147.25
0,2021-09-08,767.30,778.95,763.55,777.45,804.35,452.25,SUNPHARMA,18.21,Rising High,0,55.028264,18.507143,Unknown,Unknown,Unknown Green,Nifty 50,4.34,135.54
1,2021-09-09,789.00,793.70,784.20,790.15,819.00,400.15,AXISBANK,26.07,Rising High,1,59.787677,18.289286,Unknown,Unknown,Green Doji,Nifty 50,3.09,147.32
18,2021-09-09,489.90,492.00,487.75,491.15,496.50,325.00,BPCL,26.97,Rising High,1,67.215594,8.846429,Unknown,Unknown,Unknown Green,Nifty 50,4.49,143.96
8,2021-09-09,458.25,466.65,456.00,463.55,475.00,154.40,HINDALCO,28.10,Rising High,0,60.506464,14.560714,V Pattern,Unknown,Unknown Green,Nifty 50,5.29,148.46
16,2021-09-09,1685.90,1694.90,1682.15,1691.60,1757.00,913.05,INFY,28.99,Rising High,0,51.897730,27.842857,Unknown,Unknown,Unknown Green,Nifty 50,1.90,129.02


## Golden Crossover
`1:5` RRR

In [67]:
for name in In.data['nifty_200']:
    df = In.open_downloaded_stock(name)
    if In.has_golden_crossover(df,lookback=11):
        print(name)

HDFCLIFE
INDIAMART


## Oversold / Overbought Stocks
`RSI`

In [69]:
for name in In.data['nifty_100']:
    df = In.open_downloaded_stock(name)
    if In.get_RSI(df) < 30:
        print(name)

# Intraday

In [21]:
def intraday_whole_num_strategy():
    '''
    If Open == High or Low for a stock around 9:30, go long if Open == Low else go high only after 9:30 on 15 minutes candle
    '''
    df = pd.read_csv(join(expanduser('~'),'Downloads',f"MW-NIFTY-200-{present.day}-{calendar.month_abbr[present.month]}-{present.year}.csv"))

    df['Equal'] = df.apply(lambda row: True if (row['OPEN \n'] == row['HIGH \n'] or row['OPEN \n'] == row['LOW \n']) else False, axis=1)
    df = df.loc[df["Equal"] == True,:]
    df['Whole'] = df['OPEN \n'].apply(lambda x:float(x.replace(',','')).is_integer())
    df = df.loc[df['Whole'] == True,:]

    result = {'Long':[], 'Short':[]}
    for index in df.index:
        open_ = df.loc[index,'OPEN \n']
        high =  df.loc[index,'HIGH \n']
        low =  df.loc[index,'LOW \n']
        ltp = df.loc[index,'LTP \n']
        name = df.loc[index,'SYMBOL \n']

        result['Long'].append(name) if open_ == low else result['Short'].append(name)
        
    return result


def NR7(name):
    '''
    If Current "DAILY" candle has the lowest Range ( High - Low) from the previous 6 candles, go SHORT on next ( 8th candle) if it breaks the low or go LONG if it breaks the high
    args:
        name: name of the stock
    '''
    df = In.open_downloaded_stock(name)

    min_range = int(df.loc[0,'HIGH']  - df.loc[0,'LOW']) # Assume the smallest range is for current day
    for index in df.index[1:7]:
        if (int(df.loc[index,'HIGH']  - df.loc[index,'LOW'])) <= min_range:
            return False
    return True

## `NR-7` Narrow Range 7

In [20]:
for name in In.data['nifty_50']:
    if NR7(name):
        print(name)

AXISBANK
BAJAJ-AUTO
DIVISLAB
DRREDDY
HCLTECH
HDFCLIFE
ICICIBANK
INDUSINDBK
INFY
JSWSTEEL
RELIANCE
SHREECEM
TCS
UPL
ULTRACEMCO


# Get Particulars

In [25]:
In.get_particulars('DABUR',10000,100,delta=0.005)#, entry = 620.1 , stop_loss=587.9)  # 620.1, 587.9

{'Buying Price': 619.08,
 'Stop-Loss %': 5.13,
 'Target %': 10.25,
 'Quantity': 3.0,
 'Stop-Loss Price': 587.3485,
 'Trigger Price': 682.51,
 'Risk Per Share': 31.73,
 'Profit Per Share': 63.43,
 'Max loss on this config': 95.19,
 'Max Gain on this config': 190.29,
 'Price To Profit Ratio': 9.75,
 'Index': 'Nifty 100'}